In [1]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 50.4 MB 49.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
import cv2
from keras.applications.vgg16 import preprocess_input
from keras.models import  model_from_json
from keras.utils.vis_utils import plot_model


Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    studydesc=ds.StudyDescription
    flag = True
    #check headers
    
    if ds.BodyPartExamined!='CHEST':
        print ("Incorrect body part examined in Diacom. it should be CHEST. it is instead :",ds.BodyPartExamined)
        flag=False
    if ds.Modality!='DX':
        print ("Incorrect Modality in Diacom. it should be DX. it is instead :",ds.Modality)
        flag=False
    if ds.PatientPosition not in ['AP', 'PA']:
        print ("Incorrect Patient Position in Diacom. it should be AP or PA. it is instead :",ds.PatientPosition)
        flag=False
    
    if flag :
        return img, studydesc
    else:
        return None, None
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_size): 
    # todo
    b,h,w,c=img_size
    proc_img=img.copy()
    proc_img = cv2.resize(proc_img,(h,w))
    backtorgb = cv2.cvtColor(proc_img,cv2.COLOR_GRAY2RGB)
    backtorgb=np.expand_dims(backtorgb,0)
    ret=preprocess_input(backtorgb)
    
    return ret

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    json_file = open(model_path, "r") 
    loaded_model_json = json_file.read() 
    json_file.close() 
    my_model = model_from_json(loaded_model_json) 
    my_model.load_weights(weight_path)
    
    return my_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    pred_y=model.predict(img)
    if pred_y[0][0] > thresh:
        prediction ="Pneumonia"
    else :
        prediction ="No Pneumonia"
    return prediction 


In [5]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path ="xray_class_my_model.best.hdf5"

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
#my_model = load_model(model_path,weight_path)
thresh = 0.522 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img,desc = check_dicom(i)
  
    if img is None:
        continue
    
    img_proc = preprocess_image(img,IMG_SIZE)
    
    my_model=load_model(model_path,weight_path)
    pred = predict_image(my_model,img_proc,thresh)
    print("Prediction :",pred)
    print("Actual:",desc)
    

Load file test1.dcm ...
Prediction : No Pneumonia
Actual: No Finding
Load file test2.dcm ...
Prediction : No Pneumonia
Actual: Cardiomegaly
Load file test3.dcm ...
Prediction : Pneumonia
Actual: Effusion
Load file test4.dcm ...
Incorrect body part examined in Diacom. it should be CHEST. it is instead : RIBCAGE
Load file test5.dcm ...
Incorrect Modality in Diacom. it should be DX. it is instead : CT
Load file test6.dcm ...
Incorrect Patient Position in Diacom. it should be AP or PA. it is instead : XX
